# Hardware Cordic
## python代码来源于PYNQ开源社区：https://mp.weixin.qq.com/s/7tl7EMD08Ag6iF4OzVDpGw

In [3]:
from pynq import Overlay

overlay = Overlay("./cordic.bit")

cordic = overlay.cordiccart2pol_0

In [4]:
import random
import cmath
import struct
import binascii
import numpy as np

#创建等差数组：[0, 0.1, 0.2, ... , 9.9, 10]
r_error=np.arange(0,10,0.1)
theta_error = np.arange(0,10,0.1)

#随机生成10组坐标
for i in range(1,10):
#(1) 随机生成-1到1之间的浮点数
    x = random.uniform(-1,1)
    y = random.uniform(-1,1)
#(2) 计算golden output
    cn = complex(x,y)  #复数类型书写
    cn = cmath.polar(cn) #转换成极坐标
#(3) 将输入转换成byte发送给FPGA
    x = (struct.unpack('<I',struct.pack('<f',x))[0])
    y = (struct.unpack('<I',struct.pack('<f',y))[0])
#(4) 向FPGA中写数值, 地址映射可在HLS中查看
    cordic.write(0x10,x)
    cordic.write(0x18,y)
#(5) 启动
    cordic.write(0x00,1)
    cordic.write(0x00,0)
    r = hex(cordic.read(0x20))
    r = r[2:]
    theta = hex(cordic.read(0x30))
    theta = theta[2:]
#(6) 转换成float类型
    if r != 0:
        r=struct.unpack('>f',binascii.unhexlify(r))
        r = r[0]
    if theta != 0:
        theta = struct.unpack('>f',binascii.unhexlify(theta))
        theta =theta[0]
#(7) 和golden输出进行比较
    r_error[i] = "{0:.8f}".format(abs(r-cn[0]))
    theta_error[i] = "{0:.8f}".format(abs(theta-cn[1]))

sum_sq_r=0
sum_sq_theta=0
for i in range(1,10):
    sum_sq_r = sum_sq_r +(r_error[i] * r_error[i])
    r_rnse=np.sqrt(sum_sq_r/(i+1))
    sum_sq_theta = sum_sq_theta + (theta_error[i] * theta_error[i])
    theta_rnse = np.sqrt(sum_sq_theta/(i+1))
    print("Radius RMSE:", r_rnse, "Theta RMSE", theta_rnse)
    if r_rnse<0.001 and theta_rnse < 0.001:
        print("PASS")
    else:
        print("FAIL")

Radius RMSE: 4.242640687119285e-08 Theta RMSE 6.5407377259755645e-06
PASS
Radius RMSE: 4.163331998932265e-08 Theta RMSE 1.0475918416380812e-05
PASS
Radius RMSE: 6.576473218982952e-08 Theta RMSE 1.7461816915773686e-05
PASS
Radius RMSE: 5.8991524815010506e-08 Theta RMSE 1.5864886384717665e-05
PASS
Radius RMSE: 6.519202405202648e-08 Theta RMSE 1.8585092592361942e-05
PASS
Radius RMSE: 6.750661343263234e-08 Theta RMSE 1.7281976160150206e-05
PASS
Radius RMSE: 7.607562027351469e-08 Theta RMSE 1.6175457103278412e-05
PASS
Radius RMSE: 7.446102634562892e-08 Theta RMSE 1.54788953094205e-05
PASS
Radius RMSE: 7.314369419163897e-08 Theta RMSE 1.5098999635737463e-05
PASS
